In [ ]:
#Sistema de recomendación

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [4]:
# Cargar el dataset de películas
Films = pd.read_csv("C:/Users/Fede/Documents/henry/Labs/Fede/Films.csv")

In [5]:
# Eliminamos filas con valores faltantes en las columnas relevantes para el análisis
Films.dropna(subset=['belongs_to_collection', 'genres', 'release_date', 'original_language'], inplace=True)

In [6]:
# Limpiar y estandarizar el texto para facilitar la búsqueda
Films['title'] = Films['title'].str.lower().str.strip()

In [7]:
Films['combined_features'] = (
    Films['belongs_to_collection'].astype(str) + ' ' +
    Films['genres'].astype(str) + ' ' +
    Films['release_date'].astype(str) + ' ' +
    Films['original_language'].astype(str)
)

In [8]:
# Crear la matriz de características TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(Films['combined_features'])

# Calcular la similitud del coseno utilizando el kernel lineal
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [13]:
from fuzzywuzzy import fuzz

def recomendacion(titulo):
    # Convertir el título ingresado a minúsculas y eliminar espacios en blanco
    titulo = titulo.lower().strip()

    # Realizar búsqueda difusa para encontrar el título más similar en el DataFrame
    match_scores = Films['title'].apply(lambda x: fuzz.partial_ratio(x.lower().strip(), titulo))
    best_match_index = match_scores.idxmax()

    # Obtener el índice de la película correspondiente al título más similar
    index = best_match_index

    # Calcular la similitud de la película con el resto de películas
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de las 5 películas más similares (excluyendo la película consultada)
    similar_movies_indices = [i[0] for i in sim_scores[1:6]]

    # Obtener los nombres de las películas recomendadas
    recommended_movies = Films['title'].iloc[similar_movies_indices].to_list()
    

    return recommended_movies

In [21]:
recomendaciones = recomendacion('Toy Story')
print(recomendaciones)

['toy story 2', 'toy story 3', 'a cinderella story', 'madagascar: escape 2 africa', "garfield's fun fest"]


In [22]:
recomendaciones = recomendacion('Pocahontas')
print(recomendaciones)

['ace ventura: when nature calls', 'ace ventura jr: pet detective', 'abraham', 'boogeyman', 'my girl 2']


In [24]:
recomendaciones = recomendacion('Species')
print(recomendaciones)

['alex cross', 'along came a spider', 'cross', 'made in hong kong', 'hellraiser: inferno']


In [25]:
recomendaciones = recomendacion('A Goofy Movie')
print(recomendaciones)

['halloween 5: the revenge of michael myers', 'halloween ii', 'halloween: resurrection', 'halloween: h20', 'halloween: the curse of michael myers']
